In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

# Wichtig, wenn das Notebook exportiert wird in HTML, dann werden die Grafiken auch im HTML angezeigt
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
from gitly.colab.plot import GitlyPlotter

# Instantiate the object using 'github' or 'git' for static plots
# or 'colab' for default dinamic Plotly plots
gitly = GitlyPlotter('github')

This script was made to run only on Google Colab notebooks, other environments was not tested completely
You are not in Colab envirement, make sure you already have Plotly > 4.9.0 and kaleido


**Brainstorming:**

Wähle dir ein Autokonzern deiner Wahl:

Wie steht der Autokonzern im Vergleich zu den Wettberwerbern dar?

1. Frage: Ist RapidCharge eine Standardfunktion oder eine Spezialfunktion mit der man werben kann?

2. Frage: Wie stehe ich im Vergleich zu meinem Mitbewerbern dar? 
Wie ist der Markt aufgeteilt --> Hast du nur 5 Mitbewerber, als wenn du 15 Mitbewerber?

3. Frage: PREIS
    - Preisstruktur anschauen --> Vergleich wie ordnen sich die Mitbewerbern dar?
    - Preissegmente
    - Wer sind meine direkten Konkurrenten?


4. Frage: MODELLE

    - Welche "Typen" (BodyStyle) gibt es?
    - Wer ist hier mein direkter Konkurrent?


5. Frage: Umweltfreundlichkeit (Effizienz)


6. Frage: Geschwindigkeit (TopSpeed)

7. Frage: ......


Erkenntnisse bitte aufschreiben! Was ist das Egebnis deiner Analyse und was bedeutet das für deinen
Konzern in dem du arbeitest?

## Die vorbereiteten Daten einlesen

In [3]:
df = pd.read_pickle("ecar_after_preprocessing.pkl")

In [4]:
df.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate,segment
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,5,55480,28.719999,72.449997,151.339996,2.088889,mass
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,5,30000,64.800003,45.090000,41.750000,0.925926,budget
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,5,56440,38.709999,72.400002,112.220001,1.550000,mass
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,5,68040,38.570000,74.160004,115.360001,1.555556,premium
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,4,32997,53.680000,28.559999,31.920000,1.117647,mass


In [5]:
df.loc[df.loc[:, 'Brand'] == 'Audi', :]

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate,segment
9,Audi,Q4 e-tron,6.3,180,400,193,540,Yes,AWD,Type 2 CCS,SUV,5,55000,44.439999,77.199997,104.220001,1.350000,mass
23,Audi,e-tron GT,3.5,240,425,197,850,Yes,AWD,Type 2 CCS,Sedan,4,125000,30.000000,83.724998,167.449997,2.000000,premium
30,Audi,e-tron 50 quattro,6.8,190,280,231,450,Yes,AWD,Type 2 CCS,SUV,5,67358,37.330002,64.680000,103.949997,1.607143,premium
42,Audi,e-tron Sportback 55 quattro,5.7,200,380,228,610,Yes,AWD,Type 2 CCS,SUV,5,81639,37.380001,86.639999,139.080002,1.605263,premium
50,Audi,e-tron 55 quattro,5.7,200,365,237,590,Yes,AWD,Type 2 CCS,SUV,5,79445,37.119999,86.504997,139.830002,1.616438,premium
76,Audi,Q4 Sportback e-tron,6.3,180,410,188,550,Yes,AWD,Type 2 CCS,SUV,5,57500,44.730000,77.080002,103.400002,1.341463,mass
87,Audi,e-tron Sportback 50 quattro,6.8,190,295,219,470,Yes,AWD,Type 2 CCS,SUV,5,69551,37.660000,64.605003,102.930000,1.593220,premium
90,Audi,e-tron S 55 quattro,4.5,210,320,270,510,Yes,AWD,Type 2 CCS,SUV,5,93800,37.650002,86.400002,137.699997,1.593750,premium
99,Audi,e-tron S Sportback 55 quattro,4.5,210,335,258,540,Yes,AWD,Type 2 CCS,SUV,5,96050,37.220001,86.430000,139.320007,1.611940,premium


## Wähle dir ein Autokonzern deiner Wahl

Ich habe den 'BMW' Autohersteller für die weitere Analyse ausgesucht.

In [6]:
our_brand = "BMW"
df["is_our_brand"] = df.Brand.apply(lambda x: True if x == our_brand else False)

df.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate,segment,is_our_brand
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,5,55480,28.719999,72.449997,151.339996,2.088889,mass,False
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,5,30000,64.800003,45.090000,41.750000,0.925926,budget,False
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,5,56440,38.709999,72.400002,112.220001,1.550000,mass,False
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,5,68040,38.570000,74.160004,115.360001,1.555556,premium,True
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,4,32997,53.680000,28.559999,31.920000,1.117647,mass,False


In [7]:
df.shape

(103, 19)

##  Welcher Autoproduzent stellt die meisten Elektroautos her?

In [8]:
df_brands = df.groupby(['Brand'], as_index=False).size().sort_values(by=['size', 'Brand'])

In [9]:
fig = px.bar(
    data_frame=df_brands,
    x='Brand',y='size',
    text = 'size',
    title="E-Autos Markt",
    labels={'Brand':'Autohersteller','size':'Anzahl Modelle am Markt'},
    color_continuous_scale=px.colors.sequential.Blugrn,
    color='size'
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_traces(showlegend=False)
fig.update_xaxes(categoryorder = "total ascending")
fig.update(layout_coloraxis_showscale=False)


fig.add_annotation(x='BMW', y=5,
            text="Wir sind hier",
            showarrow=True,
            arrowhead=1)

---
Top 5 E-Autoshersteller sind:

    - Tesla (13 Modelle)
    - Audi (9 Modelle)
    - Volkswagen (8 Modelle)
    - Nissan (8 Modelle)
    - Skoda (6 Modelle)
    
Der 'BMW' produziert nur 4 Elektoautos.

---

## Rapid Charge

### Ist Rapid Charge eine Standardfunktion oder ein Special Feature mit der man werben kann?

In [10]:
df_rapid_charge = df.loc[:, 'RapidCharge'].value_counts()
df_rapid_charge

Yes    98
No      5
Name: RapidCharge, dtype: int64

In [11]:
# px.pie(values=df_rapid_charge.values, names=df_rapid_charge.index, title= 'Rapid Charge Funktion',
#       color_discrete_sequence=px.colors.qualitative.Safe)


fig = go.Figure(data=[go.Pie(
    labels=df_rapid_charge.index,
    values=df_rapid_charge.values,
    marker_colors=px.colors.qualitative.Safe,
    pull=[0, 0.12])])

fig.update_layout(
    title_text="Rapid Charge Funktion")

fig.update_layout(legend_traceorder="reversed",legend_title_text='Hat eine RapidCharge Funktion?')


fig.show()

---

    --> 95.1 % von E-Autos hat die RapidCharge Funktion. Sie soll als eine standarte Funktion bertrachtet werden.

---

### Welche Automarken verfügen nicht über eine Schnellladefunktion?

In [12]:
df_rapid_charge_brands = df.groupby(['RapidCharge', 'Brand'], as_index=False, observed=True ).size()
df_rapid_charge_brands.head()

,RapidCharge,Brand,size
0,Yes,Tesla,13
1,Yes,Volkswagen,8
2,Yes,Polestar,1
3,Yes,BMW,4
4,Yes,Honda,2


In [13]:
fig = px.sunburst(
    df_rapid_charge_brands,
    path=['RapidCharge', 'Brand'],
    values='size',
    color_discrete_sequence=px.colors.qualitative.Vivid
)

fig.show()

In [14]:
fig = px.bar(df_rapid_charge_brands,
             x='Brand', y='size',
             title="RapidCharge",
             color = 'RapidCharge',
             labels={'Brand':'Bewerber','size':'Anzahl Modelle am Markt'},
             color_discrete_sequence=px.colors.qualitative.Vivid,
#              category_orders={"RapidCharge": ["No", "Yes"]},
            text = 'size')
fig.update_xaxes(categoryorder = "total ascending")
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.show()

---
    --> Nur zwei Autohersteller (Smart und Renault) produziren E-Autos ohne 'Rapid Charge' Funktion
    --> Alle 'BMW' Fahrzeuge sind mit der Schnellladefunktion ausgestattet.
    
---

## Preissegmente

### Preisstruktur. Wie ordnen sich die Mitbewerbern?

In [15]:
df_price_segment_brands = df.groupby(['Brand'], as_index=False, observed=True).size()#.sort_values('segment')
df_price_segment_brands.head()

,Brand,size
0,Tesla,13
1,Volkswagen,8
2,Polestar,1
3,BMW,4
4,Honda,2


In [16]:
fig = px.scatter(df.sort_values('PriceEuro'), x="Brand", y="PriceEuro", color= 'PriceEuro',
                labels={'Brand':'Mitbewerber','PriceEuro':'Preis in Euro'},
                color_continuous_scale=px.colors.sequential.Emrld, title = ' Preisstruktur')
fig.update_xaxes(categoryorder = "mean ascending")


fig.update_traces(marker=dict(size=12,
                              opacity = 0.65,
                              line=dict(width=1,
                                        color='grey')),
                  selector=dict(mode='markers'))

fig.add_hline(y=30000, line_dash="dot",
              annotation_text="30k", 
              annotation_position="bottom right", line_width=1)

fig.add_hline(y=60000, line_dash="dot",
              annotation_text="60k", 
              annotation_position="bottom right", line_width=1)

fig.add_shape(type="circle",
    xref="paper", yref="y",
    x0=0.62, y0=25000, x1=0.655, y1=80000,
    line_color="dimgrey",line_width=1
)

fig.add_annotation(x='BMW', y=82000,
            text="Wir sind hier",
            showarrow=True,
            arrowhead=1)

fig.update(layout_coloraxis_showscale=False)

fig.show()

### Preissegmente

In [17]:
df_price_segment = df.loc[:, 'segment'].value_counts().sort_values()

df_price_segment

budget     14
premium    32
mass       57
Name: segment, dtype: int64

In [18]:

fig = go.Figure(data=[go.Pie(
    labels=df_price_segment.index,
    values=df_price_segment.values,
    marker_colors=px.colors.qualitative.Safe,
    textinfo='label+percent',
    pull=[0, 0.05, 0])])

fig.update_layout(
    title_text="Preissegmente")

fig.add_annotation(text="* budget - bis 30 Tsd. Euro",
                  xref="paper", yref="paper",
                  x=0.96, y=0.08, showarrow=False)
fig.add_annotation(text="mass - 30-60 Tsd. Euro",
                  xref="paper", yref="paper",
                  x=0.97, y=0.04, showarrow=False)
fig.add_annotation(text="premium - über 60 Tsd. Euro",
                  xref="paper", yref="paper",
                  x=1, y=0, showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

---
Der Markt für Elektroautos wurde in drei Preissegmente eingeteilt: Billigautos unter 30.000 Euro, Massenautos zwischen 30.000 und 60.000 Euro und Premiumautos über 60.000 Euro.

Nur 13,6 Prozent aller Modelle sind auf dem Markt für Billigautos.

BMW stellt die Fahrzeuge für die mass und premium Segmente her.

---

## Karosseriebauformen

### Welche "Typen" (BodyStyle) gibt es?

In [19]:
df_bodystyle = df.BodyStyle.value_counts()
df_bodystyle.head()

SUV          45
Hatchback    32
Sedan        10
Liftback      5
Cabrio        3
Name: BodyStyle, dtype: int64

In [20]:
list(df_bodystyle.index)

['SUV',
 'Hatchback',
 'Sedan',
 'Liftback',
 'Cabrio',
 'Pickup',
 'SPV',
 'MPV',
 'Station']

In [21]:
fig = px.bar(
    x = df_bodystyle.index,
    y = df_bodystyle.values,
    text = df_bodystyle.values,
    title="Karosseriebauformen am Markt",
    labels={'x':'Karosseriebauform','y':'Anzahl Modelle am Markt'})

fig.update_traces(texttemplate='%{text}', textposition='outside')

---
Die beliebtesten Karosserieformen sind SUV und Hatchback.

---

### Welche Karosseriebauformen von welchen Autoproduzenten werden hergestellt.

In [22]:
df_body_style_brands = df.groupby(['BodyStyle', 'Brand'], as_index=False, observed=True).size().sort_values('size')
df_body_style_brands.head()

,BodyStyle,Brand,size
25,SUV,BMW,1
24,SUV,Volkswagen,1
50,SPV,Renault,1
26,SUV,Peugeot,1
30,SUV,MG,1


In [23]:
fig = px.bar(df_body_style_brands.sort_values('Brand'),
             x='Brand', y='size',
             text='size',
             title="Karosseriebauformen am Markt",
             color = 'BodyStyle',
             color_discrete_sequence=px.colors.qualitative.Vivid,
             category_orders={'BodyStyle' : ['SUV', 'Hatchback', 'Sedan', 'Liftback', 'Cabrio', 'Pickup', 'SPV', 'MPV', 'Station']},
            labels={'Brand':'Mitbewerber','size':'Anzahl Modelle am Markt', 'BodyStyle': 'Karosseriebauform'})
# fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_xaxes(categoryorder='array', categoryarray=['Volvo', 'Sono', 'Polestar', 'Jaguar', 'Citroen', 'Lucid',
 'DS', 'Aiways', 'Lightyear', 'SEAT', 'CUPRA', 'Lexus', 'MG', 'Mini', 'Mazda', 'Fiat', 'Peugeot',
 'Honda', 'Smart', 'Opel', 'Hyundai', 'Mercedes', 'Byton', 'BMW', 'Ford', 'Kia', 'Renault', 'Porsche', 'Skoda',
 'Nissan', 'Volkswagen', 'Audi', 'Tesla'])

fig.show()

---

Die 'BMW' produziert Elektroautos in drei Karosserievarianten (SUV, Hatchback, Sedan)

---

In [24]:
fig = px.treemap(df_body_style_brands, path=['BodyStyle', 'Brand' ], values='size',
                 color_discrete_sequence=px.colors.qualitative.Safe,
                title="Karosseriemodell am Markt")
fig.show()

### Karosseriemodell nach Segment

In [25]:
df_segment_style = df.groupby(['segment', 'BodyStyle'], as_index=False, observed=True).size()
df_segment_style.head()

,segment,BodyStyle,size
0,budget,Hatchback,12
1,budget,SUV,1
2,budget,Cabrio,1
3,mass,Sedan,2
4,mass,Hatchback,19


In [26]:
fig = px.bar(df_segment_style,
             x='size', y='BodyStyle',
             title="Karosseriemodell am Markt",
             color = 'segment',
             text ='size',
             color_discrete_sequence=px.colors.qualitative.Safe,
             category_orders={'segment' : ['budget', 'mass', 'premium']},
            labels={'segment':'Segment','size':'Anzahl Modelle am Markt', 'BodyStyle': 'Karosseriemodell'}
            )

fig.update_yaxes(categoryorder = "total descending")
fig.update_traces(texttemplate='%{text}', textposition='inside')

fig.show()

In [27]:
fig = px.bar(df_segment_style, y="BodyStyle", x="size", facet_col="segment", 
       category_orders={"segment": ["budget", "mass", "premium"],
                        'BodyStyle' : ['SUV', 'Hatchback', 'Sedan', 'Liftback', 'Cabrio', 'Pickup', 'SPV', 'MPV', 'Station']},
       color_discrete_sequence=px.colors.qualitative.Safe,
       labels={'Brand':'Bewerber','size':'Anzahl', 'segment':'seg.', "BodyStyle": "" },
            title = 'Karosseriemodell nach Segment', width=950, height=500)

fig.show()

In [28]:
# fig = px.scatter(df_segment_style, y="BodyStyle", x="segment", size='size', text="size",
#                  size_max=28,#color="segment",
#                 labels={'BodyStyle':'Body Style','segment':'Preissegment'},
#                 title = 'Body Styles vs. Preissegment')
# fig.update_traces(textposition='middle right')

# fig.show()

In [29]:
# fig = px.scatter(df, y="BodyStyle", x="PriceEuro",
#                  size_max=28,#color="segment",
#                 labels={'BodyStyle':'Body Style','PriceEuro':'Price in Euro'},
#                 title = 'Body Styles vs. Preissegment')
# fig.update_traces(textposition='middle right')

# fig.add_vline(x=30000, line_dash="dot",
#               annotation_text="30k", 
#               annotation_position="top right",
#               line_width=1)

# fig.add_vline(x=65000, line_dash="dot",
#               annotation_text="65k", 
#               annotation_position="top right",
#               line_width=1)

# fig.update_xaxes(categoryorder = "total ascending")

In [30]:
fig = make_subplots(rows=1, cols=2,shared_yaxes=True, horizontal_spacing=0.02, column_widths=[0.45, 0.55])

fig.add_trace(
    go.Scatter(
        y=df_segment_style.loc[:, "BodyStyle"],
        x=df_segment_style.loc[:,"segment"],
        mode="markers+text",
        #marker_size =df_segment_style["size"],#,
        text=df_segment_style["size"],
        textposition="middle right", 
        marker=dict(size=df_segment_style["size"]+7)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        y = df["BodyStyle"],
        x = df["PriceEuro"],
        mode="markers",
    ),
    row=1, col=2
)

fig.add_vline(x=30000, line_dash="dot",
              annotation_text="30k", 
              annotation_position="top right",
              line_width=1,  row=1, col=2)

fig.add_vline(x=60000, line_dash="dot",
              annotation_text="0k", 
              annotation_position="top right",
              line_width=1,  row=1, col=2)

fig.update_layout(height=500, width=950, title_text="Karosseriemodell vs. Preissegment")
fig.update_layout(showlegend=False)


fig.update_xaxes(title_text="Preissegment", row=1, col=1)
fig.update_xaxes(title_text="Price in Euro", row=1, col=2)

fig.update_yaxes(title_text="Karosseriemodell", row=1, col=1)

fig.show()

---
Die meisten Farhzeuge im Gudget-Segment haben Hatchback Karosseriebauform.

Im Massensegment sind Hatchback- und SUV-Karosserien am weitesten verbreitet.

Im Premiumsegment sind SUV und Sedan die häufigsten Karosseriebauformen.

---

In [31]:
df_segment_brand_style = df.groupby(['segment', 'Brand', 'BodyStyle'], as_index=False, observed=True).size()
df_segment_brand_style.head()

,segment,Brand,BodyStyle,size
0,budget,Volkswagen,Hatchback,2
1,budget,Peugeot,Hatchback,1
2,budget,Nissan,Hatchback,1
3,budget,MG,SUV,1
4,budget,Opel,Hatchback,1


In [32]:
fig = px.bar(df_segment_brand_style,
             x="Brand", y="size",
             color="BodyStyle",
             facet_row="segment",
             category_orders={
                 "segment": ["premium", "mass", "budget"],
                 'BodyStyle' : ['SUV', 'Hatchback', 'Sedan', 'Liftback', 'Cabrio', 'Pickup', 'SPV', 'MPV', 'Station']},
             color_discrete_sequence=px.colors.qualitative.Vivid,
             labels={'Brand':'Bewerber','size':'Anzahl', 'segment':'seg.', "BodyStyle": "Karosseriemodell" },
             title = 'Karosseriemodell nach Segment')


fig.update_xaxes(categoryorder='array', categoryarray= ['MG',
'Sono', 'SEAT',  'Volvo', 'Citroen', 'DS', 'Aiways', 'Lexus','Mazda','CUPRA', 'Mini','Polestar', 
 'Jaguar','Lucid','Lightyear', 'Peugeot', 'Fiat', 'Honda', 'Smart', 'Opel','Hyundai',
 'Mercedes','Byton','BMW','Ford','Kia','Renault','Porsche','Skoda','Nissan','Volkswagen','Audi','Tesla'])


fig.add_vrect(#xref="paper",
    x0=22.4, x1=23.6,
    fillcolor="darkgrey", opacity=0.5,
    layer="below", line_width=0
)

fig.show()

In [33]:
fig = px.scatter(df, x="Brand", y="PriceEuro", color='BodyStyle',
                 size_max=28,#color="segment",
                labels={'BodyStyle':'Body Style','PriceEuro':'Price in Euro'},
                title = 'Body Styles vs. Preissegment',
                color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='middle right')

fig.add_hline(y=30000, line_dash="dot",
              annotation_text="30k", 
              annotation_position="top right",
              line_width=1)

fig.add_hline(y=60000, line_dash="dot",
              annotation_text="60k", 
              annotation_position="top right",
              line_width=1)

fig.update_xaxes(categoryorder = "total ascending")

fig.show()

## Antriebssystem (PowerTrain)

In [34]:
df_powertrain = df.PowerTrain.value_counts()
df_powertrain.head()

AWD    41
FWD    37
RWD    25
Name: PowerTrain, dtype: int64

In [35]:
fig = px.pie(values=df_powertrain.values, names=df_powertrain.index, title= 'Antriebssystem',
      color_discrete_sequence=px.colors.qualitative.Safe)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [36]:
fig = px.scatter(df, x="Brand", y="PriceEuro",
                 size_max=28, color="PowerTrain",
                labels={'PowerTrain':'PowerTrain','PriceEuro':'Price in Euro'},
                color_discrete_sequence=px.colors.qualitative.Vivid,
                title = 'PowerTrain vs. Preissegment',
                opacity =0.8)
fig.update_traces(textposition='middle right')

fig.add_hline(y=30000, line_dash="dot",
              annotation_text="30k", 
              annotation_position="top right",
              line_width=1)

fig.add_hline(y=60000, line_dash="dot",
              annotation_text="60k", 
              annotation_position="top right",
              line_width=1)

fig.update_xaxes(categoryorder = "total ascending")

fig.add_shape(type="circle",
    xref="paper", yref="y",
    x0=0.735, y0=25000, x1=0.77, y1=80000,
    line_color="dimgrey",line_width=1
)
fig.add_annotation(x='BMW', y=82000,
            text="Wir sind hier",
            showarrow=True,
            arrowhead=1)

fig.show()

----------------
Auf dem Bild wir können gut sehen, dass im Premium Segment die Fahrzeuge nur mit AWD PowerTrain hergestellt werden.

    --> 'BMW' produziert nur Autos mit dem RWD Antriebssystem. Sollen wir unsere Autos im Premium Segment auch mit AWD System ausstatten?


Beeinflusst das Antriebssystem die Parameter des Elektrofahrzeugs? (siehe Abschnitt 8.5.3)

----------------

## Übersicht der Fahrzeugeigenschaften

### Daten normalisieren

In [37]:
def column_normalization(data, column):
    # x_norm = (x - min) / (max - min)
    data.loc[:, column] = (data.loc[:, column] - data.loc[:, column].min())/(data.loc[:, column].max() - data.loc[:, column].min())

In [38]:
df_norm_fast_charge = df.loc[df.loc[:, 'RapidCharge'] == 'Yes'].copy()

num_cols = df_norm_fast_charge.select_dtypes([np.number]).columns
list(num_cols)

['AccelSec',
 'TopSpeed_KmH',
 'Range_Km',
 'Efficiency_WhKm',
 'FastCharge_KmH',
 'Seats',
 'PriceEuro',
 'Max_Ladedauer_in_min',
 'Capacity_kWh',
 'ChargingPower_kWh',
 'C_Rate']

In [39]:
column_normalization(df_norm_fast_charge, 'AccelSec')
column_normalization(df_norm_fast_charge, 'TopSpeed_KmH')
column_normalization(df_norm_fast_charge, 'Range_Km')
column_normalization(df_norm_fast_charge, 'Efficiency_WhKm')
column_normalization(df_norm_fast_charge, 'FastCharge_KmH')
column_normalization(df_norm_fast_charge, 'Max_Ladedauer_in_min')
column_normalization(df_norm_fast_charge, 'Capacity_kWh')
column_normalization(df_norm_fast_charge, 'ChargingPower_kWh')
column_normalization(df_norm_fast_charge, 'C_Rate')

df_norm_fast_charge.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate,segment,is_our_brand
0,Tesla,Model 3 Long Range Dual Motor,0.210084,0.383275,0.3500,0.337278,1.000000,Yes,AWD,Type 2 CCS,Sedan,5,55480,0.020328,0.255635,0.763062,0.932260,mass,False
1,Volkswagen,ID.3 Pure,0.663866,0.128920,0.1250,0.372781,0.103896,Yes,RWD,Type 2 CCS,Hatchback,5,30000,0.547967,0.096278,0.083855,0.190696,budget,False
2,Polestar,2,0.218487,0.303136,0.2875,0.455621,0.584416,Yes,AWD,Type 2 CCS,Liftback,5,56440,0.166423,0.255344,0.520607,0.588637,mass,False
3,BMW,iX3,0.394958,0.198606,0.2375,0.603550,0.506494,Yes,RWD,Type 2 CCS,SUV,5,68040,0.164376,0.265595,0.540068,0.592180,premium,True
4,Honda,e,0.621849,0.076655,0.0000,0.378698,0.025974,Yes,RWD,Type 2 CCS,Hatchback,4,32997,0.385347,0.000000,0.022932,0.312947,mass,False


In [40]:
df.loc[df.Brand == 'Honda', :]

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate,segment,is_our_brand
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,4,32997,53.68,28.559999,31.92,1.117647,mass,False
55,Honda,e Advance,8.3,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,4,35921,53.68,28.559999,31.92,1.117647,mass,False


### Durchschnittswerte der Parameter für jede Marke mit Rapidcharge Funktion

!!! Für die weiter Analyse nehmen wir nur die Marke, die mit der Schnellladefunktion ausgestattet werden.

In [41]:
df_norm_fast_charge["AccelSec_invert"] = 1 - df_norm_fast_charge["AccelSec"]

In [42]:
df_norm_fast_charge.columns

Index(['Brand', 'Model', 'AccelSec', 'TopSpeed_KmH', 'Range_Km',
       'Efficiency_WhKm', 'FastCharge_KmH', 'RapidCharge', 'PowerTrain',
       'PlugType', 'BodyStyle', 'Seats', 'PriceEuro', 'Max_Ladedauer_in_min',
       'Capacity_kWh', 'ChargingPower_kWh', 'C_Rate', 'segment',
       'is_our_brand', 'AccelSec_invert'],
      dtype='object')

In [43]:
# AccelSec

df_mean = df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(AccelSec_invert = ('AccelSec_invert', np.mean))
df_mean.head()

,Brand,AccelSec_invert
0,Tesla,0.832579
1,Volkswagen,0.435924
2,Polestar,0.781513
3,BMW,0.651260
4,Honda,0.428571


In [44]:
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(TopSpeed_KmH = ('TopSpeed_KmH', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(Range_Km = ('Range_Km', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(Efficiency_WhKm = ('Efficiency_WhKm', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(FastCharge_KmH = ('FastCharge_KmH', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(Max_Ladedauer_in_min = ('Max_Ladedauer_in_min', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(Capacity_kWh = ('Capacity_kWh', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(ChargingPower_kWh = ('ChargingPower_kWh', np.mean)), on="Brand")
df_mean = df_mean.merge(df_norm_fast_charge.groupby("Brand", as_index=False, observed=True).agg(C_Rate = ('C_Rate', np.mean)), on="Brand")

In [45]:
df_mean.head()

,Brand,AccelSec_invert,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate
0,Tesla,0.832579,0.423211,0.413462,0.576240,0.727273,0.228339,0.432869,0.725408,0.598609
1,Volkswagen,0.435924,0.111498,0.185156,0.395710,0.301948,0.340396,0.152266,0.254865,0.380968
2,Polestar,0.781513,0.303136,0.287500,0.455621,0.584416,0.166423,0.255344,0.520607,0.588637
3,BMW,0.651260,0.172474,0.185937,0.434911,0.344156,0.278188,0.168625,0.316935,0.441877
4,Honda,0.428571,0.076655,0.000000,0.378698,0.025974,0.385347,0.000000,0.022932,0.312947


### Mean Werte am Markt und für jedes Preissegment

In [46]:
def get_mean_wert_row(data, row_name):
    return {'Brand': row_name, 'AccelSec_invert' : data.loc[:, 'AccelSec_invert'].mean(), 
                'TopSpeed_KmH' : data.loc[:, 'TopSpeed_KmH'].mean(),
               'Range_Km' : data.loc[:, 'Range_Km'].mean(),
               'Efficiency_WhKm' : data.loc[:, 'Efficiency_WhKm'].mean(),
               'FastCharge_KmH' : data.loc[:, 'FastCharge_KmH'].mean(),
                'Max_Ladedauer_in_min' : data.loc[:, 'Max_Ladedauer_in_min'].mean(),
               'Capacity_kWh' : data.loc[:, 'Capacity_kWh'].mean(),
                'ChargingPower_kWh' : data.loc[:, 'ChargingPower_kWh'].mean(),
                'C_Rate' : data.loc[:, 'C_Rate'].mean()}

In [47]:
df_mean

,Brand,AccelSec_invert,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate
0,Tesla,0.832579,0.423211,0.413462,0.576240,0.727273,0.228339,0.432869,0.725408,0.598609
1,Volkswagen,0.435924,0.111498,0.185156,0.395710,0.301948,0.340396,0.152266,0.254865,0.380968
2,Polestar,0.781513,0.303136,0.287500,0.455621,0.584416,0.166423,0.255344,0.520607,0.588637
3,BMW,0.651260,0.172474,0.185937,0.434911,0.344156,0.278188,0.168625,0.316935,0.441877
4,Honda,0.428571,0.076655,0.000000,0.378698,0.025974,0.385347,0.000000,0.022932,0.312947
5,Lucid,0.941176,0.442509,0.550000,0.449704,0.584416,0.463586,0.473178,0.516765,0.248383
6,Peugeot,0.478992,0.094077,0.115625,0.402367,0.298701,0.176221,0.096045,0.250511,0.571827
7,Audi,0.708683,0.268293,0.233333,0.713346,0.516595,0.158526,0.295251,0.608684,0.614797
8,Mercedes,0.613445,0.175377,0.225000,0.686391,0.285714,0.411768,0.279923,0.340440,0.307034
9,Nissan,0.535714,0.156794,0.197656,0.536982,0.292208,0.345935,0.210146,0.307794,0.357882


In [48]:
pd.DataFrame(get_mean_wert_row(df_mean, 'Markt_mean'), index=[0])

,Brand,AccelSec_invert,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate
0,Markt_mean,0.541431,0.161828,0.193884,0.46768,0.294099,0.368215,0.182299,0.281054,0.378573


In [49]:
df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_mean, 'Markt_mean'), index=[0])],
    ignore_index=True, axis=0)


In [50]:
df_norm_budget = df_norm_fast_charge.loc[df_norm_fast_charge.segment == 'budget', :]

df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_budget, 'Budget_mean'), index=[0])],
    ignore_index=True, axis=0)

In [51]:
df_norm_mass = df_norm_fast_charge.loc[df_norm_fast_charge.segment == 'mass', :]

df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_mass, 'Mass_mean'), index=[0])],
    ignore_index=True, axis=0)

In [52]:
df_norm_premium = df_norm_fast_charge.loc[df_norm_fast_charge.segment == 'premium', :]

df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_premium, 'Premium_mean'), index=[0])],
    ignore_index=True, axis=0)

In [53]:
df_mean.tail()

,Brand,AccelSec_invert,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,Max_Ladedauer_in_min,Capacity_kWh,ChargingPower_kWh,C_Rate
31,Fiat,0.420168,0.094077,0.100000,0.378698,0.207792,0.264990,0.078281,0.168702,0.441978
32,Markt_mean,0.541431,0.161828,0.193884,0.467680,0.294099,0.368215,0.182299,0.281054,0.378573
33,Budget_mean,0.342857,0.062369,0.078125,0.371006,0.115584,0.440509,0.059115,0.092358,0.296918
34,Mass_mean,0.519208,0.150697,0.192522,0.454459,0.311224,0.357986,0.178923,0.291933,0.392876
35,Premium_mean,0.773634,0.342661,0.328320,0.644786,0.559659,0.263715,0.363435,0.605522,0.520062


### Durchschnittliche Parameterwerte in Abhängigkeit von der Antriebssystem

In [54]:
df_norm_awd = df_norm_fast_charge.loc[df_norm_fast_charge.PowerTrain == 'AWD', :]
df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_awd, 'AWD_mean'), index=[0])],
    ignore_index=True, axis=0)

df_norm_fwd = df_norm_fast_charge.loc[df_norm_fast_charge.PowerTrain == 'FWD', :]
df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_fwd, 'FWD_mean'), index=[0])],
    ignore_index=True, axis=0)

df_norm_rwd = df_norm_fast_charge.loc[df_norm_fast_charge.PowerTrain == 'RWD', :]
df_mean = pd.concat(
    [df_mean, pd.DataFrame(get_mean_wert_row(df_norm_rwd, 'RWD_mean'), index=[0])],
    ignore_index=True, axis=0)

### Visualisirung

In [55]:
params_df = df_mean.melt(id_vars=["Brand"], 
        var_name="Parameter", 
        value_name="Value")

params_df

,Brand,Parameter,Value
0,Tesla,AccelSec_invert,0.832579
1,Volkswagen,AccelSec_invert,0.435924
2,Polestar,AccelSec_invert,0.781513
3,BMW,AccelSec_invert,0.651260
4,Honda,AccelSec_invert,0.428571
...,...,...,...
346,Mass_mean,C_Rate,0.392876
347,Premium_mean,C_Rate,0.520062
348,AWD_mean,C_Rate,0.531905
349,FWD_mean,C_Rate,0.297612


#### BMW Autos im Vergleich mit direkten Mitbewerbern

In [56]:
df_competitors_test = params_df.loc[params_df.loc[:, 'Brand'].isin([our_brand, 'Tesla', "Ford", "Mercedes", "Byton", "Skoda", "Volkswagen", "Nissan"])]

In [57]:
df_competitors_test

,Brand,Parameter,Value
0,Tesla,AccelSec_invert,0.832579
1,Volkswagen,AccelSec_invert,0.435924
3,BMW,AccelSec_invert,0.651260
8,Mercedes,AccelSec_invert,0.613445
9,Nissan,AccelSec_invert,0.535714
...,...,...,...
320,Mercedes,C_Rate,0.307034
321,Nissan,C_Rate,0.357882
327,Skoda,C_Rate,0.351987
340,Ford,C_Rate,0.241663


In [58]:
fig = px.line_polar(df_competitors_test, r="Value", theta="Parameter", color="Brand", line_close=True,
            color_discrete_sequence=px.colors.qualitative.Vivid,
            title="BMW Autos im Vergleich zu direkten Mitbewerbern",
            labels={'Brand':'Mitbewerber'})
fig.show()

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a

#### BMW Fahrzeuge im Vergleich zu den technischen Durchschnittswerten nach dem Preissegment

In [59]:
df_test = params_df.loc[params_df.loc[:, 'Brand'].isin([our_brand, "Budget_mean", "Mass_mean", "Premium_mean", "Markt_mean"])]

In [60]:
fig = px.line_polar(df_test, r="Value", theta="Parameter", color="Brand", line_close=True,
            color_discrete_sequence=px.colors.qualitative.Vivid,
            title="BMW im Vergleich zu Durchschnittswerten nach Preissegment",
            labels={'Brand':''})
fig.show()

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a

#### Technischen Durchschnittswerten nach dem Antriebssystem

In [61]:
df_powertrain_test = params_df.loc[params_df.loc[:, 'Brand'].isin(["AWD_mean", "FWD_mean", "RWD_mean"])]
df_powertrain_test

,Brand,Parameter,Value
36,AWD_mean,AccelSec_invert,0.765731
37,FWD_mean,AccelSec_invert,0.405229
38,RWD_mean,AccelSec_invert,0.537015
75,AWD_mean,TopSpeed_KmH,0.327186
76,FWD_mean,TopSpeed_KmH,0.091560
77,RWD_mean,TopSpeed_KmH,0.157956
114,AWD_mean,Range_Km,0.320579
115,FWD_mean,Range_Km,0.126563
116,RWD_mean,Range_Km,0.208036
153,AWD_mean,Efficiency_WhKm,0.608024


In [62]:
fig = px.line_polar(df_powertrain_test, r="Value", theta="Parameter", color="Brand", line_close=True,
            color_discrete_sequence=px.colors.qualitative.Vivid,
            title="Durchschnittswerten nach Antriebssystem",
            labels={'Brand':'Antriebssystem'})
fig.show()

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\natalie\anaconda3\lib\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Im Diagramm ist ein klarer Zusammenhang zwischen der Art der Übertragung und der technischen Leistung zu erkennen.

## Höchstgeschwindigkeit der Topmodelle

In [63]:
fig = px.bar_polar(df.loc[df.loc[:, 'segment'] == 'premium'], r="TopSpeed_KmH", color = 'PriceEuro', theta="Model", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()